In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

The csv data file provided contains synthetic data representing a stratified random sample of customers who have a savings account, and who satisfy several other criteria: they are *all aged 18 or over*, *none of them have retired yet*, and all of them have a current account (based on separate analysis) appears to be their main current account. The data are for a single year only.  All monetary amounts are in GBP and are full-year amounts. 

In [2]:
df = pd.read_csv('data/savings_customers_data_v1.csv')

```
cust_unique_id	A unique ID generated for each customer for the purposes of this analysis, in order to preserve customer anonymity.
age	The age of each customer in years at the mid-point of the year for which data was queried. 
gender	The self-identified gender recorded by each customer.
ethnicity_group	The self-identified etnnicity group recorded by each customer. 
geo_region	The geographical region of the UK to which the customers recorded address and postcode correspond. 
net_salary	The total inflows into the customer's current account identified as consisting of salary payments, based on a separate analysis. 
other_income	The total inflows into the customer's current account identified as consisting of non-salary forms of income, based on a separate analysis. 
observed_income	The sum of net_salary and other_income. 
housing_spend	The total outflows from the customer's current account identified as consisting of mortgage, rent and other housing costs, based on a separate analysis. 
childcare_spend	The total outflows from the customer's current account identified as consisting of nursery, childminder and other childcare costs, based on a separate analysis. 
gambling_spend	The total outflows from the customer's current account identified as consisting of spending on gambling websites, based on a separate analysis. 
observed_surplus	The total of observed income minus the total of observed costs. 
credit_score	The credit score assigned to the customer by an internal bank credit scoring process. 
savings_bal_lbg	The total balance of savings held by the customer with Lloyds Banking Group entities as of the start of the year for which data was queried. 
annual_net_savings_lbg	The total of savings payments made by the customer into savings accounts with Lloyds Banking Group entities, minus withdrawls from those accounts, during the year for which data was queried.
```

### 1.	Cleanse the data and prepare it to be suitable for statistical modelling and machine learning. Comment on the steps performed and explain the rationale for them.

In [3]:
df.info() # check data type and null counts 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30305 entries, 0 to 30304
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cust_unique_id          30305 non-null  int64  
 1   age                     30305 non-null  int64  
 2   gender                  30305 non-null  object 
 3   ethnicity_group         30305 non-null  object 
 4   geo_region              30305 non-null  object 
 5   net_salary              30305 non-null  float64
 6   other_income            26067 non-null  float64
 7   observed_income         30305 non-null  float64
 8   housing_spend           22949 non-null  float64
 9   childcare_spend         30305 non-null  float64
 10  gambling_spend          30305 non-null  float64
 11  observed_surplus        30305 non-null  float64
 12  credit_score            30305 non-null  object 
 13  savings_bal_lbg         30305 non-null  float64
 14  annual_net_savings_lbg  30305 non-null

In [4]:
print(f'other_income has {30305-26067} null rows')
print(f'housing_spend has {30305-22949} null rows')

other_income has 4238 null rows
housing_spend has 7356 null rows


In [5]:
# check if we have any duplicate customers 

print('unique customers', len(df), df.cust_unique_id.nunique())

unique customers 30305 30305


In [6]:
# check the genders 
print('unique genders', df.gender.unique())

unique genders ['M' 'Other' 'F' '9999']


In [7]:
# how many customers have 9999 for gender?
df[df.gender == '9999'].cust_unique_id.count()

360

In [8]:
# check ethnicity_group
print('ethnicity_group', df.ethnicity_group.unique())

ethnicity_group ['White British' 'Afro-Carribbean' 'White Other' 'South Asian' 'Other']


In [9]:
# check geo_region
print('geo_region', df.geo_region.unique())

geo_region ['Southeast' 'Midlands' 'Scotland' 'Southwest' 'Wales' 'North']


In [10]:
# check why credit_score is object-type. Trying to cast it to float-type
# string-types will show up as an exception 
def isfloat(num):
    try: 
        float(num)
        return 0
    except:
        return num
    
df.credit_score.apply(isfloat).unique()

array([0, 'PROMO34'], dtype=object)

In [11]:
# how customers with PROMO34 credit score?
df[df.credit_score == 'PROMO34'].cust_unique_id.count()

1050

In [12]:
# check if any customers credit score is 0
df[df.credit_score == 0].cust_unique_id.count()

0

Reasonable assumptions:

1. Keep gender == 9999 in its own group, assumption here that we don't know the gender information of these customers
2. In the absense of any background knowlege about 'PROMO34' assume these customers don't have a credit score, i.e. set the credit score to 0 for these customers. 

In [13]:
df.credit_score.replace(to_replace='PROMO34', value='0', inplace=True)

In [14]:
# change dtype to float
df.credit_score = df.credit_score.astype(float)

In [15]:
df.describe() # statistics of the numberical values

,cust_unique_id,age,net_salary,other_income,observed_income,housing_spend,childcare_spend,gambling_spend,observed_surplus,credit_score,savings_bal_lbg,annual_net_savings_lbg
count,30305.00000,30305.000000,30305.000000,26067.000000,30305.000000,22949.000000,30305.000000,30305.000000,30305.000000,30305.000000,3.030500e+04,30305.000000
mean,15153.00000,41.654182,56585.867472,13580.886176,69246.199349,24719.267675,593.375517,98.408160,51270.967701,1748.052575,8.405509e+05,21299.618218
std,8748.44429,15.444812,11794.832398,5862.046413,14708.576183,4325.260951,2528.447257,2881.362877,16050.305701,399.942384,7.323375e+05,10018.424977
min,1.00000,4.000000,-1000.000000,0.000000,32518.450000,10573.570000,0.000000,0.000000,-408635.510000,0.000000,0.000000e+00,0.000000
25%,7577.00000,29.000000,50407.530000,9962.795000,59196.380000,21605.140000,0.000000,0.000000,40242.040000,1643.360000,1.514217e+05,16891.130000
50%,15153.00000,42.000000,56643.520000,12716.980000,67338.610000,24603.170000,0.000000,0.000000,50093.010000,1792.250000,7.113819e+05,22870.700000
75%,22729.00000,55.000000,64805.760000,18749.150000,82136.110000,28155.700000,0.000000,0.000000,60232.450000,1986.020000,1.397923e+06,28352.660000
max,30305.00000,68.000000,104374.080000,29752.890000,131050.070000,42189.020000,17778.930000,448664.260000,128700.410000,2719.650000,3.615059e+06,42951.370000


Observations:

1. Age observed in the dataset doesn't match with the expectations, i.e. the customers are at least 18 years old 
2. Since age of retirement could vary - assuming customers with age 68 are not retired
3. Negative net salary - most likely wrong data 
4. Negative observed surplus

In [16]:
# lets check the data points with negative observed surplus

df[df.observed_surplus < 0]

,cust_unique_id,age,gender,ethnicity_group,geo_region,net_salary,other_income,observed_income,housing_spend,childcare_spend,gambling_spend,observed_surplus,credit_score,savings_bal_lbg,annual_net_savings_lbg
5520,5521,56,F,White British,Midlands,52684.51,8485.29,61169.80,21141.05,0.0,448664.26,-408635.51,1270.0,0.0,15461.96
7737,7738,29,M,White British,Southwest,52658.90,11081.95,63740.85,23087.18,0.0,55459.84,-14806.17,1270.0,0.0,8333.24
9127,9128,39,F,White British,Scotland,49068.74,6664.48,55733.22,18989.86,0.0,79359.71,-42616.35,1270.0,0.0,7542.20
11975,11976,30,F,White British,Scotland,47230.98,10475.23,57706.21,25984.46,0.0,71718.62,-39996.88,1270.0,0.0,10734.64
19593,19594,56,M,White British,Scotland,60877.46,11932.78,72810.24,23404.60,0.0,121227.12,-71821.48,1270.0,0.0,15574.38
25309,25310,43,F,White British,Southeast,61547.49,19374.34,80921.83,25531.50,0.0,106260.16,-50869.84,1270.0,0.0,13666.38


6 cases with large gambling spenditure and interestingly credit score for all of them is 127. But do the numbers add up? check down the line

In [17]:
# check the cases with negative net salary 
df[df.net_salary < 0]

,cust_unique_id,age,gender,ethnicity_group,geo_region,net_salary,other_income,observed_income,housing_spend,childcare_spend,gambling_spend,observed_surplus,credit_score,savings_bal_lbg,annual_net_savings_lbg
28,29,27,F,White British,Midlands,-1000.0,9686.01,57227.30,22189.57,0.0,535.95,34501.79,1569.80,64201.14,16440.68
109,110,40,M,South Asian,Scotland,-1000.0,NaN,50617.41,23329.22,0.0,0.00,39395.17,1699.93,566555.04,18165.15
183,184,39,F,Other,Southwest,-1000.0,0.00,41964.94,NaN,0.0,0.00,41964.94,1424.65,341405.99,12950.55
265,266,31,M,White British,Scotland,-1000.0,9062.41,63348.90,25410.71,0.0,0.00,37938.18,1684.19,220003.09,19727.81
327,328,37,M,South Asian,Southeast,-1000.0,16375.54,74808.21,NaN,0.0,0.00,74808.21,1886.64,587868.96,23703.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29986,29987,32,F,White British,Southeast,-1000.0,18329.32,83331.75,27848.25,0.0,0.00,55483.49,2029.34,378943.31,26222.88
30089,30090,19,M,White British,Southeast,-1000.0,16953.39,82102.91,28734.20,0.0,0.00,53368.71,1926.88,0.00,24369.25
30097,30098,60,M,White British,Scotland,-1000.0,11077.95,70223.62,23089.74,0.0,0.00,47133.88,1795.13,1583330.70,32886.18
30193,30194,34,M,White British,Southeast,-1000.0,19080.87,84917.11,NaN,0.0,0.00,84917.11,2004.54,502142.99,21348.40


In [18]:
# are all the rows -1000? 
df[df.net_salary < 0].net_salary.unique()

array([-1000.])

508 cases of negative net salary, all -1000. Could be coming from a bug. Will drop these cases 

In [19]:
# check the number of customers with age below 18? 
df[df.age < 18].cust_unique_id.count()

1166

In [20]:
# check the number of customers with negative net salary who are at least 18 years old 
df.loc[(df['net_salary'] < 0) & (df['age'] > 17)].cust_unique_id.count()

486

Apply the following requirements on the datasets:

1. the customers are at least 18 years old 
2. The sum of net salary and other income equals observed income 
3. Observed surplus equals to the observed income minus any outflow due to housing, childcare and gambling 

These requirements will provide a high degree of confidence on the input datasets

In [21]:
df_reduced = df.loc[(df['net_salary'] + df['other_income'] == df['observed_income']) & (df['age'] > 17) & (df['observed_income'] - df['housing_spend'] - df['childcare_spend'] - df['gambling_spend'] == df['observed_surplus']) ]

Explore the reduced dataset 

In [22]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4973 entries, 7 to 30304
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cust_unique_id          4973 non-null   int64  
 1   age                     4973 non-null   int64  
 2   gender                  4973 non-null   object 
 3   ethnicity_group         4973 non-null   object 
 4   geo_region              4973 non-null   object 
 5   net_salary              4973 non-null   float64
 6   other_income            4973 non-null   float64
 7   observed_income         4973 non-null   float64
 8   housing_spend           4973 non-null   float64
 9   childcare_spend         4973 non-null   float64
 10  gambling_spend          4973 non-null   float64
 11  observed_surplus        4973 non-null   float64
 12  credit_score            4973 non-null   float64
 13  savings_bal_lbg         4973 non-null   float64
 14  annual_net_savings_lbg  4973 non-null   floa

In [23]:
df_reduced.describe()

,cust_unique_id,age,net_salary,other_income,observed_income,housing_spend,childcare_spend,gambling_spend,observed_surplus,credit_score,savings_bal_lbg,annual_net_savings_lbg
count,4973.000000,4973.000000,4973.000000,4973.000000,4973.000000,4973.000000,4973.000000,4973.000000,4973.000000,4973.000000,4.973000e+03,4973.000000
mean,14914.927609,42.295194,56706.284166,12957.427860,69663.712027,24266.592765,437.416586,142.692306,44817.010370,1733.270046,8.273920e+05,21142.132986
std,8783.101164,14.770439,9228.206831,6121.905148,14922.026869,4424.060356,2172.023481,6374.438481,13086.130567,394.979663,7.174805e+05,9970.967264
min,8.000000,18.000000,32936.650000,0.000000,32936.650000,10573.570000,0.000000,0.000000,-408635.510000,0.000000,0.000000e+00,0.000000
25%,7209.000000,29.000000,50062.750000,9614.500000,60156.140000,21195.450000,0.000000,0.000000,37747.510000,1632.710000,1.589339e+05,16775.570000
50%,14710.000000,42.000000,55853.960000,12087.550000,68003.390000,24070.560000,0.000000,0.000000,44356.230000,1770.610000,6.962334e+05,22560.020000
75%,22460.000000,55.000000,63729.070000,18436.290000,82285.550000,27798.340000,0.000000,0.000000,53702.510000,1963.780000,1.365427e+06,28260.370000
max,30305.000000,68.000000,104374.080000,26902.960000,130957.700000,41872.550000,16643.490000,448664.260000,90235.230000,2712.580000,3.361906e+06,40629.580000


In [24]:
# still negative surplus?, lets check
df_reduced[df_reduced.observed_surplus < 0]

,cust_unique_id,age,gender,ethnicity_group,geo_region,net_salary,other_income,observed_income,housing_spend,childcare_spend,gambling_spend,observed_surplus,credit_score,savings_bal_lbg,annual_net_savings_lbg
5520,5521,56,F,White British,Midlands,52684.51,8485.29,61169.8,21141.05,0.0,448664.26,-408635.51,1270.0,0.0,15461.96


In [25]:
# Interesting outlier: negative surplus due to large gambling spend but significant net savings at the end of the year 
# Since this is a single outlier, for modeling purposes drop it 

df_reduced = df_reduced[df_reduced.observed_surplus > 0]

In [26]:
df_reduced.describe()

,cust_unique_id,age,net_salary,other_income,observed_income,housing_spend,childcare_spend,gambling_spend,observed_surplus,credit_score,savings_bal_lbg,annual_net_savings_lbg
count,4972.000000,4972.000000,4972.000000,4972.000000,4972.000000,4972.000000,4972.000000,4972.000000,4972.000000,4972.000000,4.972000e+03,4972.000000
mean,14916.816975,42.292438,56707.093051,12958.327325,69665.420376,24267.221394,437.504562,52.482820,44908.211601,1733.363222,8.275584e+05,21143.275418
std,8782.973809,14.770645,9228.958672,6122.192235,14923.041334,4424.283189,2172.233079,405.537788,11397.771521,394.964727,7.174567e+05,9971.644618
min,8.000000,18.000000,32936.650000,0.000000,32936.650000,10573.570000,0.000000,0.000000,7213.590000,0.000000,0.000000e+00,0.000000
25%,7213.500000,29.000000,50062.247500,9617.897500,60156.092500,21195.540000,0.000000,0.000000,37748.005000,1632.950000,1.589503e+05,16778.210000
50%,14716.500000,42.000000,55860.985000,12088.395000,68004.405000,24071.190000,0.000000,0.000000,44356.850000,1770.715000,6.962408e+05,22560.290000
75%,22462.250000,55.000000,63729.532500,18436.797500,82286.857500,27799.017500,0.000000,0.000000,53703.850000,1963.862500,1.365463e+06,28260.575000
max,30305.000000,68.000000,104374.080000,26902.960000,130957.700000,41872.550000,16643.490000,18652.720000,90235.230000,2712.580000,3.361906e+06,40629.580000


Note: The statistics of the dataset is reduced significantly. It could be possible to correct some of the input datasets to preserve larger statistics if background knowledge on various separate analyses and methods that are used to derive these datasets is present and can be accessed. 

However, since these are synthetic data sampled randomly from stratified dataset, we can always sample more with the above requirements applied.

In [27]:
# encode the categorical variables and insert in the dataframe
le = {}
cat_columns = ['gender', 'ethnicity_group', 'geo_region']
for column in cat_columns:
    index = df_reduced.columns.get_loc(column)
    le[column] = LabelEncoder()
    sel_column = df_reduced[column]
    le[column].fit(sel_column)
    en_col_name = column + '_cat'
    en_col = le[column].transform(sel_column)
    df_reduced.insert(index+1, en_col_name, en_col, False)
    print(df_reduced.groupby(column)[en_col_name].first())

    



gender
9999     0
F        1
M        2
Other    3
Name: gender_cat, dtype: int64
ethnicity_group
Afro-Carribbean    0
Other              1
South Asian        2
White British      3
White Other        4
Name: ethnicity_group_cat, dtype: int64
geo_region
Midlands     0
North        1
Scotland     2
Southeast    3
Southwest    4
Wales        5
Name: geo_region_cat, dtype: int64


In [28]:
# save the cleaned and processed data in a pickle file for next stages of the analysis
df_reduced.to_pickle('data/customer_data_modified.pkl')